In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [3]:
## load the dataset
data = pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
## preprocessing the data
## dropping irrelevant data columns 

data = data.drop(['RowNumber','Surname','CustomerId'],axis=1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
## converting categorical values to numerical values
## encoding gender
label_gender_encoder = LabelEncoder()
data['Gender'] = label_gender_encoder.fit_transform(data['Gender'])
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [ ]:
    ## one hot encoding the geography column
    from sklearn.preprocessing import OneHotEncoder
    ohe_geography = OneHotEncoder()

    geo_encoder = ohe_geography.fit_transform(data[['Geography']])
    # ohe_geography.get_feature_names_out(['Geography']) ## to show the features 

In [7]:
## converting the ohe values to dataframes to fit the given dataset table
geo_encoded_columns = pd.DataFrame(geo_encoder.toarray(),columns=ohe_geography.get_feature_names_out(['Geography']))
geo_encoded_columns

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [8]:
## combining the ohe columns with the original dataset
data = pd.concat([data.drop(['Geography'],axis=1),geo_encoded_columns],axis=1)
data.head()


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [9]:
## saving the gender encoder and ohe_geography to pickle file for future use
with open('label_encode_gender.pkl','wb') as file:
    pickle.dump(label_gender_encoder,file)

with open('onehotencoder_geo.pkl','wb') as file:
    pickle.dump(ohe_geography,file)

In [10]:
## Divide the dataset into independent(x) and dependent(y) features
x = data.drop('Exited',axis=1)  ## features
y = data['Exited']  ## required outputs

## spliting the data into training and testing sets

xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.2,random_state=42)

##  scale these features
scaler = StandardScaler()
xtrain = scaler.fit_transform(xtrain)
xtest = scaler.transform(xtest)

In [11]:
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

## ANN IMPLEMENTATION

In [12]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [13]:
xtrain.shape ## give touple giving the dimensions of training matrix 
## [0] gives number of rows
## [1] gives number of columns

(8000, 12)

In [14]:
## Building ANN model
## Dence(no of hiddenlayer nodes, activation function, number of input features)
model = Sequential([
    Input(shape = (xtrain.shape[1],)), ## Input layer features
    Dense(64,activation='relu'), ## hidden layer 1
    Dense(32,activation='relu'), ## hidden layer 2
    Dense(1,activation='sigmoid') ## output layer
])

In [15]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [16]:
## this code is to set your own laerning rate because if not defined optimizers have 
## default values set

import tensorflow
opt = tensorflow.keras.optimizers.Adam(learning_rate = 0.01)

In [17]:
##compiling the model
## the arguments have some preset values but these values can be explicitly defined

model.compile(optimizer='adam',loss="binary_crossentropy",metrics=['accuracy'])



In [18]:
## Set up the tensorboard
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

log_dir = "logs/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir,histogram_freq=1)

In [19]:
## Setup early stopping : to stop the model training after sufficient number of epochs

early_stopping_callback = EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)

In [20]:
# Train the model

history = model.fit(xtrain,ytrain,validation_data=(xtest,ytest),epochs=100,
                    callbacks=[tensorflow_callback,early_stopping_callback])

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.7674 - loss: 0.5106 - val_accuracy: 0.8315 - val_loss: 0.3942
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8401 - loss: 0.3970 - val_accuracy: 0.8530 - val_loss: 0.3569
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8543 - loss: 0.3628 - val_accuracy: 0.8610 - val_loss: 0.3386
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8585 - loss: 0.3521 - val_accuracy: 0.8635 - val_loss: 0.3394
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8600 - loss: 0.3401 - val_accuracy: 0.8635 - val_loss: 0.3373
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8621 - loss: 0.3383 - val_accuracy: 0.8555 - val_loss: 0.3423
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8597 - loss: 0.3406 - val_accuracy: 0.8640 - val_loss: 0.3343
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8684 - loss: 0.3206 - val_acc

In [21]:
model.save('model.h5')


In [22]:
## load tensor board extension

%load_ext tensorboard


In [25]:
%tensorboard --logdir logs/fit20250123-164411/

Reusing TensorBoard on port 6007 (pid 13768), started 6:05:48 ago. (Use '!kill 13768' to kill it.)